# Agents and Tools

In [1]:
import tomli, os
with open("../.streamlit/secrets.toml","rb") as f:
    secrets = tomli.load(f)
os.environ["OPENAI_API_KEY"] = secrets["OPENAI_API_KEY"]
os.environ["SERPAPI_API_KEY"] = secrets["SERPAPI_API_KEY"]

## Smith: Simple mini independent tool handler

In [4]:
import importlib
smith = importlib.import_module('smith')
importlib.reload(smith)

<module 'smith' from 'c:\\Users\\ydebray\\Downloads\\gpt-programming-book\\chap5\\smith.py'>

In [5]:
from smith import *

In [9]:
prompt = 'Who is the CEO of Twitter?'
print(agent(prompt)) # Calling agent without passing tools

As of September 2021, the CEO of Twitter is Jack Dorsey.


In [68]:
tools = load_tools()
tools

[{'name': 'search',
  'desc': 'Useful for when you need to answer questions about current events.',
  'example': 'What movie won best picture in 2023?'},
 {'name': 'python',
  'desc': 'Useful for when performing computing and trying to solve mathematical problems',
  'example': 'What is the 10th element of the Fibonacci suite'}]

In [69]:
prompt = 'Who is the CEO of Twitter?'
res = agent(prompt,tools=tools)
print(res)

tool: search
------------------
The CEO of Twitter is Linda Yaccarino, as reported by The Verge.


Breaking down the tool choice and execution.

In [70]:
prompt = 'What movie won best picture in 2024?'
pick_tool(prompt,tools)

'search'

In [71]:
res = search_tool(prompt)
print(res)

Answer the user request given the following information retrieved from an internet search:
  Oppenheimer
  


In [72]:
prompt = 'What is the square root of 42?'
# res = pick_tool(prompt,tools)
res = agent(prompt,tools=tools)
print(res)

tool: python
------------------
import math

result = math.sqrt(42)
print(result)


## LangChain agents

In [59]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain_openai import ChatOpenAI
# Load the model
llm = ChatOpenAI(temperature=0)
# Load in some tools to use
tools = load_tools(["serpapi"])
# Finally, let's initialize an agent with:
# 1. The tools
# 2. The language model
# 3. The type of agent we want to use.

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
# Now let's test it out!
agent.run("who is the ceo of twitter?")



> Entering new AgentExecutor chain...
I should search for the current CEO of Twitter.
Action: Search
Action Input: "current CEO of Twitter"
Observation: Linda Yaccarino
Thought:That doesn't seem right, I should search again.
Action: Search
Action Input: "current CEO of Twitter 2021"
Observation: Parag Agrawal
Thought:Parag Agrawal is the current CEO of Twitter.
Final Answer: Parag Agrawal

> Finished chain.


'Parag Agrawal'

It seems like the agent is getting the date wrong.
Let's create our own tool to handle the date.

In [57]:
from langchain.agents import tool
from datetime import date

In [58]:
# Define a tool that returns the current date
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [60]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [61]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")

c:\Users\ydebray\Downloads\gpt-programming-book\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I should use the `time` tool to find out today's date.

Action:
```
{
  "action": "time",
  "action_input": ""
}
```


Observation: 2024-05-04
Thought:Final Answer: 2024-05-04

> Finished chain.


In [65]:
agent("who is the CEO of twitter today? (First get the date and then answer)")



> Entering new AgentExecutor chain...
Thought: Let's first find out today's date and then search for the current CEO of Twitter. 

Action:
```
{
  "action": "time",
  "action_input": ""
}
```
Observation: 2024-05-04
Thought:Now that we have today's date, let's search for the current CEO of Twitter. 

Action:
```
{
  "action": "Search",
  "action_input": "current CEO of Twitter 2024"
}
```
Observation: Linda Yaccarino
Thought:Final Answer: Linda Yaccarino

> Finished chain.


{'input': 'As of today, who is the CEO of twitter? (First get the date and then ask the question.)',
 'output': 'Linda Yaccarino'}

Another tool for search: DuckDuckGo

`pip install duckduckgo_search`

In [7]:
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
res = search.run("CEO of Twitter")

'Twitter framed a copy of one of her motivational tweets about "wearing 4 inch heels" while working as an executive and hung it in a dining common area in the San Francisco office. She has also ... FILE - Twitter CEO Elon Musk, center, speaks with Linda Yaccarino, chairman of global advertising and partnerships for NBC, at the POSSIBLE marketing conference, Tuesday, April 18, 2023, in Miami Beach, Fla. Musk announced Friday, May 12, 2023, that he\'s hiring Yaccarino to be the new CEO of San Francisco-based Twitter, which is now called X Corp. (AP Photo/Rebecca Blackwell, File) Updated on: May 15, 2023 / 9:49 AM EDT / MoneyWatch. Elon Musk has tapped Linda Yaccarino, until this week the head of advertising at NBCUniversal, to replace him as CEO of Twitter, he announced ... David Becker/Getty Images. Jack Dorsey cofounded Twitter in 2006 and the company made him a billionaire. He stepped down as Twitter CEO in 2021 and supported Elon Musk\'s takeover of the company ... Jack Dorsey is the

c:\Users\ydebray\Downloads\gpt-programming-book\env\lib\site-packages\curl_cffi\aio.py:204: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


## OpenAI Functions

- [OpenAI Blog function calling](https://openai.com/blog/function-calling-and-other-api-updates)
- [OpenAI Doc function calling](https://platform.openai.com/docs/guides/gpt/function-calling)
- [How_to_call_functions_with_chat_models.ipynb](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb)
- [How_to_call_functions_for_knowledge_retrieval.ipynb](https://github.com/openai/openai-cookbook/blob/0d1436b8d9b858c220d708a446a09eef54be61b0/examples/How_to_call_functions_for_knowledge_retrieval.ipynb)

In [29]:
import tomli, openai, json
with open('../.streamlit/secrets.toml','rb') as f:
    toml_dict = tomli.load(f)
openai.api_key = toml_dict['OPENAI_API_KEY']

In [10]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [48]:
import os
functions_list = os.listdir("functions")
functions_list

['get_current_weather.json']

In [53]:
functions = []
with open("functions/get_current_weather.json", "r") as f:
    functions.append(json.loads(f.read()))
functions

[{'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city and state, e.g. San Francisco, CA'},
    'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}},
   'required': ['location']}}]

In [54]:
# Step 1: send the conversation and available functions to GPT
messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response_message = response.choices[0].message
dict(response_message)

{'content': None,
 'role': 'assistant',
 'function_call': FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'),
 'tool_calls': None}

In [55]:
# Step 2: check if GPT wanted to call a function, and call it
if response_message.function_call is not None:
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple
    function_name = response_message.function_call.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
function_response

'{"location": "Boston, MA", "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}'

In [34]:
# Step 3: send the info on the function call and function response to GPT
messages.append(response_message)  # extend conversation with assistant's reply
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": function_response,
    }
)  # extend conversation with function response
second_response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)  # get a new response from GPT where it can see the function response
second_response.choices[0].message.content

'The current weather in Boston is sunny and windy with a temperature of 72 degrees.'